In [1]:
import requests

# Processing equations

## Generating pMML from LaTeX equations

In [2]:
tex_equation = "E = mc^{2}"
r = requests.post("http://eq2mml:8001/latex/mml", json={"tex_src": tex_equation})
print(r.text)

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block" alttext="E = mc^{2}">
  <mi>E</mi>
  <mo>=</mo>
  <mi>m</mi>
  <msup>
    <mi>c</mi>
    <mrow class="MJX-TeXAtom-ORD">
      <mn>2</mn>
    </mrow>
  </msup>
</math>


## Generating pMML from equation images

In [8]:
files = {
  "data": open("/data/skema/images/bayes-rule-white-bg.png", "rb"),
}
r = requests.post("http://eq2mml:8001/image/mml", files=files)
print(r.text)

<math> <mrow> <mn> <mtr> <mtd> <msub> <mi> P </mi> <mrow> <mo> ( </mo> <mrow> <mi> b </mi> </mrow> <mrow> <mo> | </mo> </mrow> <mi> a </mi> <mo> ) </mo> </mrow> </msub> <mo> = </mo> <mrow> <mfrac> <mrow> <msub> <mi> P </mi> <mrow> <mo> ( </mo> <mrow> <mi> a </mi> </mrow> <mrow> <mo> | </mo> </mrow> <mrow> <mi> b </mi> </mrow> <mo> ) </mo> </mrow> </msub> <mo> &#x22C5 </mo> <msub> <mi> P </mi> <mrow> <mo> ( </mo> <mrow> <mi> b </mi> </mrow> <mo> ) </mo> </mrow> </msub> </mrow> <mrow> <msubsup> <mi> P </mi> <mrow> <mo> ( </mo> <mrow> <mi> a </mi> </mrow> <mo> ) </mo> </mrow> <mrow> <mo> ( </mo> <mrow> <mi> b </mi> </mrow> <mo> ) </mo> </mrow> </msubsup> </mrow> </mfrac> </mrow> </math>
